In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark=SparkSession.\
    builder.\
        appName("Word Count").\
            getOrCreate()
    

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 13:28:58 INFO SparkEnv: Registering MapOutputTracker
24/05/23 13:28:58 INFO SparkEnv: Registering BlockManagerMaster
24/05/23 13:28:58 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/23 13:28:58 INFO SparkEnv: Registering OutputCommitCoordinator


In [6]:
sc=spark.sparkContext

In [8]:
sentence=spark.read.format("text").load("/user/sovik/dummy_files/*")

In [10]:
sentence.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                         

In [31]:
words_unfiltered=sentence.select(split(col("value")," ").alias("list_of_words"))

In [32]:
words_unfiltered.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|list_of_words                                                                                                      

In [33]:
words_exploded=words_unfiltered.select(explode(col("list_of_words")).alias("exploded_list"))

In [34]:
words_exploded.show(truncate=False)

+-------------+
|exploded_list|
+-------------+
|             |
|In           |
|a            |
|lush         |
|forest,      |
|under        |
|the          |
|shade        |
|of           |
|ancient      |
|oaks,        |
|lived        |
|a            |
|cunning      |
|fox          |
|and          |
|a            |
|vain         |
|crow.        |
|One          |
+-------------+
only showing top 20 rows



In [35]:
words_cleansed=words_exploded.select(regexp_extract(col("exploded_list"),"[a-zA-z]+",0 ).alias("Filtered_words"))

In [36]:
words_cleansed.show(truncate=False)

+--------------+
|Filtered_words|
+--------------+
|              |
|In            |
|a             |
|lush          |
|forest        |
|under         |
|the           |
|shade         |
|of            |
|ancient       |
|oaks          |
|lived         |
|a             |
|cunning       |
|fox           |
|and           |
|a             |
|vain          |
|crow          |
|One           |
+--------------+
only showing top 20 rows



In [38]:
#churn out out only valid strings

words_cleansed=words_cleansed.filter(col("Filtered_words")!="")

In [39]:
words_cleansed.show(truncate=False)

+--------------+
|Filtered_words|
+--------------+
|In            |
|a             |
|lush          |
|forest        |
|under         |
|the           |
|shade         |
|of            |
|ancient       |
|oaks          |
|lived         |
|a             |
|cunning       |
|fox           |
|and           |
|a             |
|vain          |
|crow          |
|One           |
|sunny         |
+--------------+
only showing top 20 rows



In [46]:
#count each words

words_k_v_pair = words_cleansed.withColumn("frequency", lit(1))

In [47]:
words_k_v_pair.show()

+--------------+---------+
|Filtered_words|frequency|
+--------------+---------+
|            In|        1|
|             a|        1|
|          lush|        1|
|        forest|        1|
|         under|        1|
|           the|        1|
|         shade|        1|
|            of|        1|
|       ancient|        1|
|          oaks|        1|
|         lived|        1|
|             a|        1|
|       cunning|        1|
|           fox|        1|
|           and|        1|
|             a|        1|
|          vain|        1|
|          crow|        1|
|           One|        1|
|         sunny|        1|
+--------------+---------+
only showing top 20 rows



In [49]:
words_k_v_pair.\
    groupBy(col("Filtered_words")).\
        agg(count(col("frequency")).alias("total_count")).\
            show()

+--------------+-----------+
|Filtered_words|total_count|
+--------------+-----------+
|          Thus|          1|
|          must|          1|
|           for|          2|
|         under|          1|
|          they|          1|
|       leaving|          1|
|            on|          1|
|      feathers|          1|
|       flatter|          1|
|          high|          1|
|         lived|          1|
|          plan|          1|
|            up|          1|
|         voice|          1|
|           are|          2|
|           The|          2|
|       ancient|          1|
|             a|          8|
|        bright|          1|
|     Flattered|          1|
+--------------+-----------+
only showing top 20 rows



In [50]:
sc.stop()